In [1]:
import numpy as np
import sys,os
sys.path.append("..")
from glob import glob
from shutil import rmtree
import pandas as pd
from natsort import natsorted
import json

In [19]:
def vistype(desc: pd.Series):
    visname = desc.strip('.png')
    visname = visname.strip('.jpg')
    json_file = os.path.join('/netpool/homes/wangyo/Dataset/VisQA/merged/qa', visname + '.json')
    json_file = open(json_file)
    visjson = json.load(json_file)
    json_file.close()
    return visjson['vistype']


base_path = '/netpool/homes/wangyo/Dataset/VisQA/eyetracking/Results'
subjects_path = glob(os.path.join(
    f"{base_path}/*/", "*", ""))
subjects_path = [os.path.abspath(path) for path in subjects_path]

good_sum = 0
good_true = 0
bad_sum = 0
bad_true = 0
for path in subjects_path:
    pname = path.split('/')[-1]
    fgood = f"{path}/{pname}_images_with_fixations_and_elems_enlarge_0.csv"
    #fgood = f"{path}/{pname}_images_with_fixations_and_elems_enlarge_0.5degree.csv"
    fbad = f"{path}/{pname}_images_with_fixations_and_elems_enlarge_0.1degree.csv"
    if(not os.path.exists(fgood) or not os.path.exists(fbad)): continue
    tmp_good=pd.read_csv(fgood)
    tmp_bad=pd.read_csv(fbad)
    tmp_good = tmp_good[tmp_good['stage'] == 'enc']
    tmp_bad = tmp_bad[tmp_bad['stage'] == 'enc']

    # apply the vistype
    #tmp_good['image_name'] = tmp_good['image_name'].apply(vistype)
    #tmp_good = tmp_good[tmp_good['image_name'] == 'scatter']
    #tmp_bad['image_name'] = tmp_bad['image_name'].apply(vistype)
    #tmp_bad = tmp_bad[tmp_bad['image_name'] == 'scatter']


    gtrue = (tmp_good['contains']==True).sum()
    gfalse = (tmp_good['contains']==False).sum()
    btrue = (tmp_bad['contains']==True).sum()
    bfalse = (tmp_bad['contains']==False).sum()
    good_sum += gtrue
    good_sum += gfalse
    good_true += gtrue
    bad_sum += btrue
    bad_sum += bfalse
    bad_true += btrue
    

print(good_sum, bad_sum)
print(good_true/good_sum)
print(bad_true/bad_sum)

105089 155890
0.5538067733064355
0.8985630893578805


In [35]:
np.tan(0.1)

0.10033467208545055

# AVG
## 0, 1.25, 2.5, 3.75, 5, 6.25, 7.5, 8.75, 10
### 0.55, 0.60, 0.65, 0.69, 0.73, 0.76, 0.79, 0.81, 0.84

# Scatter
## 0, 1.25, 2.5, 3.75, 5, 6.25, 7.5, 8.75, 10
### 0.46, 0.51, 0.57, 0.62, 0.67, 0.72, 0.76, 0.79, 0.82

# line
## 0, 1.25, 2.5, 3.75, 5, 6.25, 7.5, 8.75, 10
### 0.41, 0.45, 0.50, 0.54, 0.59, 0.62, 0.66, 0.69, 0.72

# bar
## 0, 1.25, 2.5, 3.75, 5, 6.25, 7.5, 8.75, 10
### 0.56,0.60, 0.66, 0.71, 0.75, 0.78, 0.81, 0.83, 0.86

# pie
## 0, 1.25, 2.5, 3.75, 5, 6.25, 7.5, 8.75, 10
### 0.64, 0.66, 0.68, 0.71, 0.73, 0.75, 0.77, 0.79, 0.80